# Capstone Project - Auto Accident Prediction (Week 2)
## Applied Data Science Capstone by IBM/Coursera

This notebook will be used for the Applied Data Science Capstone project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Say you are driving to another city for work or to visit some friends. It is rainy and windy. On the way to your destination, you come across a terrible traffic jam on the other side of the highway. Long lines of cars are barely moving. As you keep driving, police car start appearing from afar, shutting down the highway. There is an accident and a helicopter is transporting the ones involved in the crash to the nearest hospital. The victems must be in critical condition for all of this to be happening.
 
Now, wouldn't it be great if there is something in place that could warn you, given the weather and the road conditions, about the possibility of you getting into a car accident and how severe it would be.  The advance warning could prompt you to  drive more carefully or even change your travel plans if you are able to.

## Data <a name="data"></a>

Load the required libraries

In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
%matplotlib inline

### Retrieve The Dataset
The data used to train and evaluate the model is the collision data set from the SDOT Traffic Management Division, Traffic Records Group. The data set is updated weekly from 2004 to the present. The data set is compiled from all collisions provided by the Seattle Police department and recorded by the Traffic Records Group.


Download the current collision data from <a name=Seattle Geo Data>http://data-seattlecitygis.opendata.arcgis.com</a>

In [2]:
!wget -O Collisions.csv https://opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv

--2020-09-03 20:51:37--  https://opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv
Resolving opendata.arcgis.com (opendata.arcgis.com)... 34.235.215.225, 34.196.73.37, 3.222.165.155, ...
Connecting to opendata.arcgis.com (opendata.arcgis.com)|34.235.215.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘Collisions.csv’

    [       <=>                             ] 84,814,898  59.5MB/s   in 1.4s   

2020-09-03 20:51:39 (59.5 MB/s) - ‘Collisions.csv’ saved [84814898]



### Load Data from CSV file
The data has unlabeled extra columns, which will cause an error if not accounted for. The _OBJECTID_ is used as the index for this dataset.

In [3]:
cols = pd.read_csv('Collisions.csv', nrows=1).columns
df = pd.read_csv('Collisions.csv', usecols=cols, index_col=2)
df.head()

,X,Y,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,-122.288688,47.532714,29800,29800,1177964,Unmatched,Block,NaN,BEACON ER AVE S BETWEEN S PORTLAND ST AND S CH...,NaN,...,NaN,NaN,NaN,4315006.0,NaN,NaN,NaN,0,0,N
2,NaN,NaN,115700,115700,10097005,Unmatched,NaN,NaN,NaN,NEI,...,NaN,NaN,NaN,10097005.0,NaN,NaN,NaN,0,0,N
3,-122.355556,47.727318,1358,1358,3568600,Matched,Block,NaN,GREENWOOD AVE N BETWEEN N 134TH ST AND N 136TH ST,,...,Dry,Daylight,NaN,NaN,NaN,28,From opposite direction - one left turn - one ...,0,0,N
4,-122.317563,47.618764,70700,70700,2806057,Matched,Block,NaN,E DENNY WAY BETWEEN 11TH AVE AND 12TH AVE,NaN,...,Dry,Other,NaN,7065007.0,NaN,32,One parked--one moving,0,0,N
5,-122.361015,47.538551,53600,53600,2127310,Matched,Block,NaN,DELRIDGE WAY SW BETWEEN SW MYRTLE ST AND SW OR...,NaN,...,Dry,Daylight,NaN,6137017.0,NaN,14,From same direction - both going straight - on...,0,0,N


### Preprocess The Data

Normalize the data and fill in missing values where it makes sense. Display the frequency tables for various features to helpdetermine which features to use.

In [4]:
df['ADDRTYPE'] = df['ADDRTYPE'].fillna('Unknown')
print("\nAddress Type:\n", df['ADDRTYPE'].value_counts())

df['WEATHER'] = df['WEATHER'].fillna('Unknown')
print("\nWeather:\n", df['WEATHER'].value_counts())

df['LIGHTCOND'] = df['LIGHTCOND'].fillna('Unknown')
print("\nLight Conditions:\n", df['LIGHTCOND'].value_counts())

df['ROADCOND'] = df['ROADCOND'].fillna('Unknown')
print("\nRoad Conditions:\n", df['ROADCOND'].value_counts())

df['JUNCTIONTYPE'] = df['JUNCTIONTYPE'].fillna('Unknown')
print("\nJunction Type:\n", df['JUNCTIONTYPE'].value_counts())

# treat an blank record as N
df['INATTENTIONIND'] = df['INATTENTIONIND'].fillna('N')
print("\nInattention Indicator:\n", df['INATTENTIONIND'].value_counts())

# treat an blank record as N, a 0 as N and 1 as Y
df['UNDERINFL'] = df['UNDERINFL'].fillna('N')
df['UNDERINFL'] = df['UNDERINFL'].replace(['0','1'],['N','Y'])
print("\nUnder Influence:\n", df['UNDERINFL'].value_counts())

# treat an blank record as N, a 0 as N and 1 as Y
df['PEDROWNOTGRNT'] = df['PEDROWNOTGRNT'].fillna('N')
df['PEDROWNOTGRNT'] = df['PEDROWNOTGRNT'].replace(['0','1'],['N','Y'])
print("\nPedestrian Not Granted:\n", df['PEDROWNOTGRNT'].value_counts())

# treat an blank record as N, a 0 as N and 1 as Y
df['SPEEDING'] = df['SPEEDING'].fillna('N')
df['SPEEDING'] = df['SPEEDING'].replace(['0','1'],['N','Y'])
print("\nSpeeding:\n", df['SPEEDING'].value_counts())

print("\nHit Parked Car:\n", df['HITPARKEDCAR'].value_counts())

print("\nSeverity Code:\n", df['SEVERITYCODE'].value_counts())



Address Type:
 Block           144784
Intersection     71774
Unknown           3712
Alley              874
Name: ADDRTYPE, dtype: int64

Weather:
 Clear                       114342
Unknown                      41724
Raining                      34019
Overcast                     28504
Snowing                        919
Other                          851
Fog/Smog/Smoke                 577
Sleet/Hail/Freezing Rain       116
Blowing Sand/Dirt               56
Severe Crosswind                26
Partly Cloudy                    9
Blowing Snow                     1
Name: WEATHER, dtype: int64

Light Conditions:
 Daylight                    119149
Dark - Street Lights On      50048
Unknown                      40201
Dusk                          6074
Dawn                          2599
Dark - No Street Lights       1573
Dark - Street Lights Off      1236
Other                          244
Dark - Unknown Lighting         20
Name: LIGHTCOND, dtype: int64

Road Conditions:
 Dry               12

### Assess The Features To Use

Remove rows where where the value is _Unknown_.

In [ ]:
print("Rows before cleaning: ", df.shape)
df = df[~df['ADDRTYPE'].isin(['Unknown'])]
df = df[~df['WEATHER'].isin(['Unknown'])]
df = df[~df['LIGHTCOND'].isin(['Unknown'])]
df = df[~df['ROADCOND'].isin(['Unknown'])]
df = df[~df['SEVERITYDESC'].isin(['Unknown'])]
print ("Rows after cleaning: ", df.shape)

Rows before cleaning:  (221144, 39)
Rows after cleaning:  (174631, 39)


Remove unneeded columns/features

In [ ]:
df.drop(inplace=True, columns=['INCKEY', 'COLDETKEY','REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE', 'COLLISIONTYPE', 'PEDCOUNT', 'PEDCYLCOUNT', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'])
df.head()


,X,Y,SEVERITYDESC,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
OBJECTID,,,,,,,,
3,-122.355556,47.727318,Property Damage Only Collision,2,2,Overcast,Dry,Daylight
4,-122.317563,47.618764,Property Damage Only Collision,2,2,Clear,Dry,Other
5,-122.361015,47.538551,Property Damage Only Collision,2,2,Clear,Dry,Daylight
6,-122.386772,47.564720,Injury Collision,4,2,Clear,Dry,Daylight
7,-122.365869,47.525967,Property Damage Only Collision,2,2,Clear,Dry,Dark - Street Lights On


In [ ]:
df.shape

(174631, 8)

### Convert Categorical Features to Numeric Values

In [ ]:
Feature = df[['PERSONCOUNT','VEHCOUNT']]
# do One Hot Encoding
Feature = pd.concat([Feature,pd.get_dummies(df['WEATHER'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['ROADCOND'])], axis=1)
Feature = pd.concat([Feature,pd.get_dummies(df['LIGHTCOND'])], axis=1)
Feature.head()

,PERSONCOUNT,VEHCOUNT,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,...,Standing Water,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other
OBJECTID,,,,,,,,,,,,,,,,,,,,,
3,2,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,4,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,2,2,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
X = Feature
y = df['SEVERITYDESC'].values

X= preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.


Train set: (139704, 28) (139704,)
Test set: (34927, 28) (34927,)


## Methodology <a name="methodology"></a>

## K Nearest Neighbor (KNN)

In [ ]:
Ks = 10
mean_accKNN = np.zeros((Ks-1))
mean_jacKNN = np.zeros((Ks-1))
mean_F1KNN  = np.zeros((Ks-1))
for n in range(1,Ks):
    
    #Train Model and Predict
    print ("Ks = ", n)
    neigh = KNeighborsClassifier(n_neighbors=n, n_jobs=1, weights='distance').fit(X_train,y_train)
    yhat=neigh.predict(X_test)

    mean_accKNN[n-1] = metrics.accuracy_score(y_test, yhat)
    mean_jacKNN[n-1] = jaccard_similarity_score(y_test, yhat)
    mean_F1KNN[n-1]  = f1_score(y_test, yhat, average='weighted')
    
print ("KNN Accuracy table: ", mean_accKNN)
print( "The best accuracy is", mean_accKNN.max(), "with k=", mean_accKNN.argmax()+1) 

print ("KNN Jaccard index table: ", mean_jacKNN)
print( "The best Jaccard index is", mean_jacKNN.max(), "with k=", mean_jacKNN.argmax()+1) 

print ("KNN F1-score table: ", mean_F1KNN)
print( "The best F1-score is", mean_F1KNN.max(), "with k=", mean_F1KNN.argmax()+1) 

Ks =  1
Ks =  2
Ks =  3


In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeClassifier(criterion="entropy", max_depth = 10)
DT_model.fit(X_train,y_train)
DT_model

Ks = 10
mean_accDT = np.zeros((Ks-1))
mean_jacDT = np.zeros((Ks-1))
mean_F1DT  = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict
    print ("Ks = ", n)
    DT_model = DecisionTreeClassifier(criterion="entropy", max_depth = n)
    DT_model.fit(X_train,y_train)
    yhat = DT_model.predict(X_test)

    mean_accDT[n-1] = metrics.accuracy_score(y_test, yhat)
    mean_jacDT[n-1] = jaccard_similarity_score(y_test, yhat)
    mean_F1DT[n-1]  = f1_score(y_test, yhat, average='weighted')
    
print ("Decision Tree Accuracy table: ", mean_accDT)
print( "The best accuracy is", mean_accDT.max(), "with k=", mean_accDT.argmax()+1) 

print ("Decision Tree Jaccard index table: ", mean_jacDT)
print( "The best Jaccard index is", mean_jacDT.max(), "with k=", mean_jacDT.argmax()+1) 

print ("Decision Tree F1-score table: ", mean_F1DT)
print( "The best F1-score is", mean_F1DT.max(), "with k=", mean_F1DT.argmax()+1) 

In [ ]:
from sklearn import svm
SVM_model = svm.SVC()
SVM_model.fit(X_train, y_train) 

In [ ]:
yhat = SVM_model.predict(X_test)
print ("The SVM model accuracy is: ", metrics.accuracy_score(y_test, yhat))
print("SVM Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("SVM F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )


In [ ]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(C=0.01).fit(X_train,y_train)
LR_model

In [ ]:
yhat = LR_model.predict(X_test)
print ("The LR model accuracy is: ", metrics.accuracy_score(y_test, yhat))
yhat_prob = LR_model.predict_proba(X_test)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("LR F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(y_test, yhat_prob))

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>